## Scala比较器：Ordered与Ordering
在项目中，我们常常会遇到排序（或比较）需求，比如：对一个Person类
```scala
case class Person(name: String, age: Int) {
  override def toString = {
    "name: " + name + ", age: " + age
  }
}
```
按name值逆词典序、age值升序做排序；在Scala中应如何实现呢？

### 两个特质
Scala提供两个特质（trait）Ordered与Ordering用于比较。其中，Ordered混入（mix）Java的Comparable接口，而Ordering则混入Comparator接口。众所周知，在Java中

* 实现Comparable接口的类，其对象具有了可比较性；
* 实现comparator接口的类，则提供一个外部比较器，用于比较两个对象。

Ordered与Ordering的区别与之相类似：

* Ordered特质定义了相同类型间的比较方式，但这种内部比较方式是单一的；
* Ordered则是提供比较器模板，可以自定义多种比较方式。

以下源码分析基于Scala 2.11。

#### Ordered
Ordered特质更像是rich版的Comparable接口，除了compare方法外，更丰富了比较操作（<, >, <=, >=）;


此外，Ordered对象提供了从T到Ordered[T]的隐式转换（隐式参数为Ordering[T]）;

```scala
package scala
package math

import scala.language.implicitConversions

/** A trait for data that have a single, natural ordering.  See
 *  [[scala.math.Ordering]] before using this trait for
 *  more information about whether to use [[scala.math.Ordering]] instead.
 *
 *  Classes that implement this trait can be sorted with
 *  [[scala.util.Sorting]] and can be compared with standard comparison operators
 *  (e.g. > and <).
 *
 *  Ordered should be used for data with a single, natural ordering (like
 *  integers) while Ordering allows for multiple ordering implementations.
 *  An Ordering instance will be implicitly created if necessary.
 *
 *  [[scala.math.Ordering]] is an alternative to this trait that allows multiple orderings to be
 *  defined for the same type.
 *
 *  [[scala.math.PartiallyOrdered]] is an alternative to this trait for partially ordered data.
 *
 *  For example, create a simple class that implements `Ordered` and then sort it with [[scala.util.Sorting]]:
 *  {{{
 *  case class OrderedClass(n:Int) extends Ordered[OrderedClass] {
 *  	def compare(that: OrderedClass) =  this.n - that.n
 *  }
 *
 *  val x = Array(OrderedClass(1), OrderedClass(5), OrderedClass(3))
 *  scala.util.Sorting.quickSort(x)
 *  x
 *  }}}
 *
 *  It is important that the `equals` method for an instance of `Ordered[A]` be consistent with the
 *  compare method. However, due to limitations inherent in the type erasure semantics, there is no
 *  reasonable way to provide a default implementation of equality for instances of `Ordered[A]`.
 *  Therefore, if you need to be able to use equality on an instance of `Ordered[A]` you must
 *  provide it yourself either when inheriting or instantiating.
 *
 *  It is important that the `hashCode` method for an instance of `Ordered[A]` be consistent with
 *  the `compare` method. However, it is not possible to provide a sensible default implementation.
 *  Therefore, if you need to be able compute the hash of an instance of `Ordered[A]` you must
 *  provide it yourself either when inheriting or instantiating.
 *
 *  @see [[scala.math.Ordering]], [[scala.math.PartiallyOrdered]]
 *  @author  Martin Odersky
 *  @version 1.1, 2006-07-24
 */
trait Ordered[A] extends Any with java.lang.Comparable[A] {

  /** Result of comparing `this` with operand `that`.
   *
   * Implement this method to determine how instances of A will be sorted.
   *
   * Returns `x` where:
   *
   *   - `x < 0` when `this < that`
   *
   *   - `x == 0` when `this == that`
   *
   *   - `x > 0` when  `this > that`
   *
   */
  def compare(that: A): Int

  /** Returns true if `this` is less than `that`
    */
  def <  (that: A): Boolean = (this compare that) <  0

  /** Returns true if `this` is greater than `that`.
    */
  def >  (that: A): Boolean = (this compare that) >  0

  /** Returns true if `this` is less than or equal to `that`.
    */
  def <= (that: A): Boolean = (this compare that) <= 0

  /** Returns true if `this` is greater than or equal to `that`.
    */
  def >= (that: A): Boolean = (this compare that) >= 0

  /** Result of comparing `this` with operand `that`.
    */
  def compareTo(that: A): Int = compare(that)
}

object Ordered {
  /** Lens from `Ordering[T]` to `Ordered[T]` */
  implicit def orderingToOrdered[T](x: T)(implicit ord: Ordering[T]): Ordered[T] =
    new Ordered[T] { def compare(that: T): Int = ord.compare(x, that) }
}

```

#### Ordering
Ordering，内置函数Ordering.by与Ordering.on进行自定义排序：
Ordering源代码比较多，里面定义了好多关于隐式转换的代码，不一一列举了
```scala
package scala
package math

import java.util.Comparator
import scala.language.{implicitConversions, higherKinds}

/** Ordering is a trait whose instances each represent a strategy for sorting
	* instances of a type.
	*
	* Ordering's companion object defines many implicit objects to deal with
	* subtypes of AnyVal (e.g. Int, Double), String, and others.
	*
	* To sort instances by one or more member variables, you can take advantage
	* of these built-in orderings using Ordering.by and Ordering.on:
	*
	* {{{
	* import scala.util.Sorting
	* val pairs = Array(("a", 5, 2), ("c", 3, 1), ("b", 1, 3))
	*
	* // sort by 2nd element
	* Sorting.quickSort(pairs)(Ordering.by[(String, Int, Int), Int](_._2))
	*
	* // sort by the 3rd element, then 1st
	* Sorting.quickSort(pairs)(Ordering[(Int, String)].on(x => (x._3, x._1)))
	* }}}
	*
	* An Ordering[T] is implemented by specifying compare(a:T, b:T), which
	* decides how to order two instances a and b. Instances of Ordering[T] can be
	* used by things like scala.util.Sorting to sort collections like Array[T].
	*
	* For example:
	*
	* {{{
	* import scala.util.Sorting
	*
	* case class Person(name:String, age:Int)
	* val people = Array(Person("bob", 30), Person("ann", 32), Person("carl", 19))
	*
	* // sort by age
	* object AgeOrdering extends Ordering[Person] {
	*   def compare(a:Person, b:Person) = a.age compare b.age
	* }
	* Sorting.quickSort(people)(AgeOrdering)
	* }}}
	*
	* This trait and scala.math.Ordered both provide this same functionality, but
	* in different ways. A type T can be given a single way to order itself by
	* extending Ordered. Using Ordering, this same type may be sorted in many
	* other ways. Ordered and Ordering both provide implicits allowing them to be
	* used interchangeably.
	*
	* You can import scala.math.Ordering.Implicits to gain access to other
	* implicit orderings.
	*
	* @author Geoffrey Washburn
	* @version 0.9.5, 2008-04-15
	* @since 2.7
	* @see [[scala.math.Ordered]], [[scala.util.Sorting]]
	*/
@annotation.implicitNotFound(msg = "No implicit Ordering defined for ${T}.")
trait Ordering[T] extends Comparator[T] with PartialOrdering[T] with Serializable {
	outer =>

	/** Returns whether a comparison between `x` and `y` is defined, and if so
		* the result of `compare(x, y)`.
		*/
	def tryCompare(x: T, y: T) = Some(compare(x, y))

	/** Returns an integer whose sign communicates how x compares to y.
		*
		* The result sign has the following meaning:
		*
		*  - negative if x < y
		*  - positive if x > y
		*  - zero otherwise (if x == y)
		*/
	def compare(x: T, y: T): Int

	/** Return true if `x` <= `y` in the ordering. */
	override def lteq(x: T, y: T): Boolean = compare(x, y) <= 0

	/** Return true if `x` >= `y` in the ordering. */
	override def gteq(x: T, y: T): Boolean = compare(x, y) >= 0

	/** Return true if `x` < `y` in the ordering. */
	override def lt(x: T, y: T): Boolean = compare(x, y) < 0

	/** Return true if `x` > `y` in the ordering. */
	override def gt(x: T, y: T): Boolean = compare(x, y) > 0

	/** Return true if `x` == `y` in the ordering. */
	override def equiv(x: T, y: T): Boolean = compare(x, y) == 0

	/** Return `x` if `x` >= `y`, otherwise `y`. */
	def max(x: T, y: T): T = if (gteq(x, y)) x else y

	/** Return `x` if `x` <= `y`, otherwise `y`. */
	def min(x: T, y: T): T = if (lteq(x, y)) x else y

	/** Return the opposite ordering of this one. */
	override def reverse: Ordering[T] = new Ordering[T] {
		override def reverse = outer

		def compare(x: T, y: T) = outer.compare(y, x)
	}

	/** Given f, a function from U into T, creates an Ordering[U] whose compare
		* function is equivalent to:
		*
		* {{{
		* def compare(x:U, y:U) = Ordering[T].compare(f(x), f(y))
		* }}}
		*/
	def on[U](f: U => T): Ordering[U] = new Ordering[U] {
		def compare(x: U, y: U) = outer.compare(f(x), f(y))
	}
}

object Ordering extends LowPriorityOrderingImplicits {
	def apply[T](implicit ord: Ordering[T]) = ord

	def by[T, S](f: T => S)(implicit ord: Ordering[S]): Ordering[T] =
		fromLessThan((x, y) => ord.lt(f(x), f(y)))
}
```

#### 官方源代码中例子运行

In [1]:
import scala.util.Sorting
val pairs = Array(("a", 5, 2), ("a", 3, 1), ("b", 1, 1))

// sort by 2nd element
Sorting.quickSort(pairs)(Ordering.by[(String, Int, Int), Int](_._2))
println(pairs.mkString(" | "))

// sort by the 3rd element, then 1st
Sorting.quickSort(pairs)(Ordering[(Int, String)].on(x => (x._3, x._1)))
println(pairs.mkString(" | "))

//def apply[T](implicit ord: Ordering[T]) = ord
//调用的是伴生对象的apply方法
val or = Ordering[(Int, String)]
println(or)

case class Person(name: String, age: Int)
val people = Array(Person("bob", 30), Person("ann", 32), Person("carl", 19))

// sort by age
object AgeOrdering extends Ordering[Person] {
	def compare(a: Person, b: Person) = a.age compare b.age
}
Sorting.quickSort(people)(AgeOrdering)
println(people.mkString(" | "))

println("*" * 20)

case class OrderedClass(n: Int) extends Ordered[OrderedClass] {
	def compare(that: OrderedClass) = this.n - that.n
}

val x = Array(OrderedClass(1), OrderedClass(5), OrderedClass(3))
scala.util.Sorting.quickSort(x)
println(x.mkString(" | "))

(b,1,1) | (a,3,1) | (a,5,2)
(a,3,1) | (b,1,1) | (a,5,2)
scala.math.Ordering$$anon$11@2df5b868
Person(carl,19) | Person(bob,30) | Person(ann,32)
********************
OrderedClass(1) | OrderedClass(3) | OrderedClass(5)


import scala.util.Sorting

pairs: Array[(String, Int, Int)] = Array(("a", 3, 1), ("b", 1, 1), ("a", 5, 2))
or: Ordering[(Int, String)] = scala.math.Ordering$$anon$11@2df5b868
defined class Person
people: Array[Person] = Array(Person("carl", 19), Person("bob", 30), Person("ann", 32))
defined object AgeOrdering
defined class OrderedClass
x: Array[OrderedClass] = Array(OrderedClass(1), OrderedClass(3), OrderedClass(5))

### 实战

In [6]:
case class Person(name: String, age: Int) {
  override def toString = {
    "name: " + name + ", age: " + age
  }
}

defined class Person

#### 比较
对于Person类，如何做让其对象具有可比较性呢？我们可使用Ordered对象的函数orderingToOrdered做隐式转换，但还需要组织一个Ordering[Person]的隐式参数

In [4]:
implicit object PersonOrdering extends Ordering[Person] {
  override def compare(p1: Person, p2: Person): Int = {
    p1.name == p2.name match {
      case false => -p1.name.compareTo(p2.name)
      case _ => p1.age - p2.age
    }
  }
}
 
val p1 = new Person("rain", 13)
val p2 = new Person("rain", 14)
import Ordered._
p1 < p2 // True

defined object PersonOrdering
p1: Person = Person("rain", 13)
p2: Person = Person("rain", 14)
import Ordered._

res3_4: Boolean = true

#### Collection Sort
在实际项目中，我们常常需要对集合进行排序。回到开篇的问题——如何对Person类的集合做指定排序呢？下面用List集合作为demo，探讨在scala集合排序。首先，我们来看看List的sort函数：
```scala
// scala.collection.SeqLike
 
def sortWith(lt: (A, A) => Boolean): Repr = sorted(Ordering fromLessThan lt)
 
def sortBy[B](f: A => B)(implicit ord: Ordering[B]): Repr = sorted(ord on f)
 
def sorted[B >: A](implicit ord: Ordering[B]): Repr = {
...
}
```

若调用sorted函数做排序，则需要指定Ordering隐式参数：

In [7]:
val p1 = new Person("rain", 24)
val p2 = new Person("rain", 22)
val p3 = new Person("Lily", 15)
val list = List(p1, p2, p3)
 
implicit object PersonOrdering extends Ordering[Person] {
  override def compare(p1: Person, p2: Person): Int = {
    p1.name == p2.name match {
      case false => -p1.name.compareTo(p2.name)
      case _ => p1.age - p2.age
    }
  }
}

list.sorted // res3: List[Person] = List(name: rain, age: 22, name: rain, age: 24, name: Lily, age: 15)

p1: Person = name: rain, age: 24
p2: Person = name: rain, age: 22
p3: Person = name: Lily, age: 15
list: List[Person] = List(name: rain, age: 24, name: rain, age: 22, name: Lily, age: 15)
defined object PersonOrdering
res6_5: List[Person] = List(name: rain, age: 22, name: rain, age: 24, name: Lily, age: 15)

若使用sortBy，也需要指定Ordering隐式参数(和sorted的区别是调用的是转换后对象的Ordering)：

In [8]:
list.sortBy[Person](t => t)

res7: List[Person] = List(name: rain, age: 22, name: rain, age: 24, name: Lily, age: 15)

In [9]:
list.sortBy(t => t)

res8: List[Person] = List(name: rain, age: 22, name: rain, age: 24, name: Lily, age: 15)

In [10]:
list.sortBy(t => t.name)

res9: List[Person] = List(name: Lily, age: 15, name: rain, age: 24, name: rain, age: 22)

若使用sortWith，则需要定义返回值为Boolean的比较函数：

In [11]:
list.sortWith { (p1: Person, p2: Person) =>
   p1.name == p2.name match {
     case false => -p1.name.compareTo(p2.name) < 0
     case _ => p1.age - p2.age < 0
   }
}
// res4: List[Person] = List(name: rain, age: 22, name: rain, age: 24, name: Lily, age: 15)

res10: List[Person] = List(name: rain, age: 22, name: rain, age: 24, name: Lily, age: 15)

#### 上界中Ordered的使用

In [12]:
/**
	*  <:                   　#表示的是Scala泛型中的上界，相当于Java泛型中的"<T extends Comparable>"
	* TT<: Ordered[T]          #表示T实现Ordered接口
	*/
class ComparableGeneralObject[T<: Ordered[T]](a:T,b:T){
	/**
		* @return     : 返回比较大的数值
		*/
	def bigger = {
		if (a > b){
			a
		}else{
			b
		}
	}
}

/**
	* 改类需要实现Ordered特质
	*/
class TeacherOrdered(val name:String,val age:Int) extends Ordered[TeacherOrdered] {
	/**
		* 重写比较的方法，比较方法按照年龄来比较
		*/
	override def compare(that: TeacherOrdered): Int = {
		this.age - that.age
	}
	/**
		* 重写toString方法
		*/
	override def toString: String = {
		this.name + "	" + this.age
	}
}

defined class ComparableGeneralObject
defined class TeacherOrdered

In [13]:
val t1 = new TeacherOrdered("丹尼斯·里奇", 70)
val t2 = new TeacherOrdered("Linus Benedict Torvalds", 49)

t1: TeacherOrdered = 丹尼斯·里奇	70
t2: TeacherOrdered = Linus Benedict Torvalds	49

In [14]:
new ComparableGeneralObject(t1,t2).bigger

res13: TeacherOrdered = 丹尼斯·里奇	70

#### 上下文界定中的Ordering

In [15]:
//方式1
//说明：
//1. [T: Ordering] 泛型
//2. obj1: T, obj2: T 接受T类型的对象
//3. implicit comparetor: Ordering[T] 是一个隐式参数
class CompareComm4[T: Ordering](obj1: T, obj2: T)(implicit comparetor: Ordering[T]) {
  def geatter = if (comparetor.compare(obj1, obj2) > 0) obj1 else obj2
}

//方式2
//方式2,将隐式参数放到方法内
class CompareComm5[T: Ordering](o1: T, o2: T) {
  def geatter = {
    def f1(implicit cmptor: Ordering[T]) = cmptor.compare(o1, o2) //返回一个数字
    //如果f1返回的值>0,就返回o1,否则返回o2
    if (f1 > 0) o1 else o2
  }
  def lowwer = {
    def f1(implicit cmptor: Ordering[T]) = cmptor.compare(o1, o2) //返回一个数字
    //如果f1返回的值>0,就返回o1,否则返回o2
    if (f1 > 0) o2 else o1
  }
}

//方式3
//方式3,使用implicitly语法糖，最简单(推荐使用)
class CompareComm6[T: Ordering](o1: T, o2: T) {
  def geatter = {
    //这句话就是会发生隐式转换，获取到隐式值 personComparetor
    //底层仍然使用编译器来完成绑定(赋值的)工作
    val comparetor = implicitly[Ordering[T]]
    println("comparetor hashcode=" + comparetor.hashCode())
    if (comparetor.compare(o1, o2) > 0) o1 else o2
  }
}

defined class CompareComm4
defined class CompareComm5
defined class CompareComm6

In [16]:
//一个普通的Person类
class Person4(val name: String, val age: Int) {

  //重写toStirng
  override def toString = this.name + "\t" + this.age
}

defined class Person4

In [17]:
//这里我定义一个隐式值  Ordering[Person]类型
implicit val personComparetor = new Ordering[Person4] {
  override def compare(p1: Person4, p2: Person4): Int =
    p1.age - p2.age
}

personComparetor: Object with Ordering[Person4] = $sess.cmd16Wrapper$Helper$$anon$1@7b464deb

In [22]:
val p1 = new Person4("mary", 30)
val p2 = new Person4("smith", 35)
val compareComm4 = new CompareComm4(p1, p2)
println(compareComm4.geatter) // "smith", 35

val compareComm5 = new CompareComm5(p1, p2)
println(compareComm5.geatter) // "smith", 35

val compareComm6 = new CompareComm6(p1, p2)
println(compareComm6.geatter) // "smith", 35

smith	35
smith	35
comparetor hashcode=2068205035
smith	35


p1: Person4 = mary	30
p2: Person4 = smith	35
compareComm4: CompareComm4[Person4] = $sess.cmd14Wrapper$Helper$CompareComm4@4670d403
compareComm5: CompareComm5[Person4] = $sess.cmd14Wrapper$Helper$CompareComm5@5157f32d
compareComm6: CompareComm6[Person4] = $sess.cmd14Wrapper$Helper$CompareComm6@7cc8cfd

#### 基于元组多字段自定义排序  

In [24]:
val pairs = Array(("a", 5, 2), ("a", 3, 1), ("b", 1, 1)) 

pairs: Array[(String, Int, Int)] = Array(("a", 5, 2), ("a", 3, 1), ("b", 1, 1))

In [25]:
//按第三个字段升序，第一个字段降序，注意，排序的字段必须和后面的tuple对应  
pairs.sortBy(r => (r._3, r._1))( Ordering.Tuple2(Ordering.Int, Ordering.String.reverse) )

res24: Array[(String, Int, Int)] = Array(("b", 1, 1), ("a", 3, 1), ("a", 5, 2))

In [26]:
//sortBy返回排序后的集合。原集合未改变
pairs

res25: Array[(String, Int, Int)] = Array(("a", 5, 2), ("a", 3, 1), ("b", 1, 1))

In [27]:
//按第三个字段降序，第一个字段降序，注意，排序的字段必须和后面的tuple对应  
pairs.sortBy(r => (r._3, r._1))( Ordering.Tuple2(Ordering.Int.reverse, Ordering.String.reverse) )

res26: Array[(String, Int, Int)] = Array(("a", 5, 2), ("b", 1, 1), ("a", 3, 1))